In [69]:
# import necessary imports
import numpy as np
from numpy_ml.neural_nets.optimizers import Adam

ImportError: cannot import name 'Hashable' from 'collections' (C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\collections\__init__.py)

import numpy as np

In [70]:
def get_batch(input, batch_size,chunk_size):

    input_batch = []
    print(type(input_batch))
    target_batch = []
    idx = np.random.randint(0,len(input)-(chunk_size+1),size=batch_size)
    for i in range(0,len(idx)-1):
        input_batch.append(input[idx[i]:idx[i]+chunk_size])
        target_batch.append(input[idx[i]+1:idx[i]+(chunk_size+1)])
    
    input_batch = np.array(input_batch)
    target_batch = np.array(target_batch)

    return input_batch, target_batch

In [ ]:
with open (r"Shakespeare_byte.txt", 'r') as f:
  shakespeare_byte_train = eval(f.read())

with open (r"vocab_train.txt", 'r') as f:
  vocab_train = eval(f.read())

vocab = vocab_train
indices = np.arange(1,len(vocab)+1,1)
inidces = indices.astype(int)
indices = indices.tolist()
key_byte = dict(zip(vocab, indices))
value_byte = dict(zip(indices,vocab))

# Map each token in shakespeare_byte_train to its index using key_byte
indices_translation = [key_byte[token] for token in shakespeare_byte_train if token in key_byte]

with open('indices_text.txt', 'w') as indices_text:
    indices_text.write(str(indices_translation))

with open (r"indices_text.txt", 'r') as f:
  indices_text = eval(f.read())

bytes_translation = [value_byte[token] for token in indices_text if token in value_byte]

with open('bytes_text.txt', 'w') as bytes_text:
    bytes_text.write(str(bytes_translation))


In [74]:
with open (r"indices_text.txt", 'r') as f:
  indices_text = eval(f.read())
  
x,y = get_batch(indices_text,32,8)
print(x.shape)
print(x)
print(y.shape)
print(y)

<class 'list'>
(31, 8)
[[ 177  698 1261  445  488  992  807 1273]
 [ 377  875  808 1263  808 1263  150  446]
 [ 439  640 1262   39 1273 1166   98 1263]
 [1534  498  270 1263  207   54 1170 1380]
 [   3  207   54  508 1263  440  240  791]
 [ 618  799  459  324  787 1419  649 1259]
 [1401 1484 1540 1263  804 1285  215  796]
 [1534  498 1284  398  256 1293  480  458]
 [1090  793  607  990 1261  766  517  438]
 [ 785 1273 1285  304  440 1263  102  785]
 [ 643  791  733 1457 1273  934 1389 1121]
 [ 188  453 1263  800  198 1273   13 1057]
 [ 797  796 1287 1391 1397 1098 1267 1263]
 [1423 1522 1207  800 1087 1396 1261  806]
 [ 177   38  181  445  122 1263  449  307]
 [ 438  863  777  797   77  868 1260 1261]
 [1293  445  828 1522 1365 1259  113 1263]
 [ 802 1273 1534 1510  171 1263  841 1065]
 [1259  105  438   85  797  976 1512  761]
 [ 307  797  445  745  442 1273   17  193]
 [1383 1531 1521 1261  439 1421 1300  462]
 [ 400  792  791 1169  805 1416 1510 1259]
 [ 867  929  802  439  894 1381

In [77]:
def decode_characters(input):
 #Decodes a list of indices back to their corresponding characters
 #given the abive defined vocabulary
    decoded = [] #given the input, we will decode it back to characters
    for i in range(0,len(input)):
        decoded.append(value_byte[input[i]])#using the translation dctionary: value_byte
#make its prettier by joining list to actual words and replacing underscores with spaces
    decoded = ''.join(decoded)
    decoded = decoded.replace('_', ' ')
    return decoded

In [78]:
text1 = [438, 1006, 1434, 853, 797, 27, 439, 1, 1024, 1130, 781, 886, 1184, 1519, 1259, 221, 27, 5, 25, 1294]

decoded = decode_characters(text1)
print(decoded)

the tragedy of antony and cleopatra dramatis personae mark antony octavius caesar m 


In [10]:
class neural_embedding:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.token_embedding_table = np.random.rand(vocab_size, vocab_size)

    def calculate_softmax(self, x):
        """Takes input array x and returns softmax."""
        soft_x = np.exp(x - np.max(x))
        softer_x = soft_x / np.sum(soft_x)
        return softer_x

    def calculate_cross_entropy(self, y_hatless, y_hat):
        """
        Takes target (y_hatless) and prediction (y_hat) and computes cross entropy loss.
        """
        # get vocab_size
        _, _, vocab_size = y_hat.shape        
        y_hat = y_hat.reshape(y_hat.shape[0]*y_hat.shape[1], y_hat.shape[2])
        y_hatless_flat = y_hatless.reshape(-1)
        # one-hot encode targets
        y_hatless_hot = np.eye(vocab_size)[y_hatless_flat]
       
        y_hat = self.calculate_softmax(y_hat)
    
        return -np.sum(y_hatless_hot*np.log(y_hat))
    
    def forward(self, idx, targets=None):
        """
        Implements forward pass with an unnecessary logitte function 
        which i only did not delete because now I'm emotionally attached.
        Args:
            idx(np.array): (B,T) numpy array of integers
            targets(np.array): (B,T) numpy array of integers
        Returns:
            input_logits(np.array)
            sometimes also: targets(np.array)
        """
        batch_size, chunk_size = idx.shape
        logits = np.zeros((batch_size, chunk_size, (self.token_embedding_table[0].size)))

        def logitte(batch_size, chunk_size, input):
            for batch in range(batch_size):
                for chunk in range(chunk_size):
                    # (B,T,C) b=batch_size, t="time"=chunk_size, c=vocab_size
                    logits[batch][chunk] = self.token_embedding_table[input[batch][chunk]]
                    
            return logits

        input_logits = logitte(batch_size, chunk_size, idx)
        
        if targets is not None:
            loss = self.calculate_cross_entropy(targets, input_logits)

            return input_logits, loss

        return input_logits

    
    def backward(self, targets, input_logits):
        # need to do the same reshaping as we did for cross entropy, apparently
        targets_flat = targets.reshape(-1)
        one_hot_targets = np.eye(self.vocab_size)[targets_flat]

        # shape after: ((batch_size*chunk_size), vocab_size)
        input_logits_2d = input_logits.reshape(input_logits.shape[0]*input_logits.shape[1], input_logits.shape[2])

        # somehow this is supposedly the combiantion of the derivative of softmax with the derivative of the CCE
        delta = one_hot_targets - input_logits_2d

        # want shape (80,80) for matrix multiplication, but with correct indices (use one-hot targets for that)
        delta_indexed = np.dot(one_hot_targets.transpose(),delta)

        # compute gradient for weight matrix: dot product between the transpose of the to layer and delta vector computed above
        gradient = (self.token_embedding_table @ delta_indexed) 

    
        return gradient

    

In [11]:
def initialize_adam(parameters) :
    """
    Initializes v and s as two python dictionaries with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.
    
    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters["W" + str(l)] = Wl
                    parameters["b" + str(l)] = bl
    
    Returns: 
    v -- python dictionary that will contain the exponentially weighted average of the gradient.
                    v["dW" + str(l)] = ...
                    v["db" + str(l)] = ...
    s -- python dictionary that will contain the exponentially weighted average of the squared gradient.
                    s["dW" + str(l)] = ...
                    s["db" + str(l)] = ...

    """
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialization of v, s. Input: "parameters". Outputs: "v, s".
    for l in range(L):
        v["dW" + str(l+1)] = np.zeros((parameters["W" + str(l+1)]).shape)
        v["db" + str(l+1)] = np.zeros((parameters["b" + str(l+1)]).shape)
        s["dW" + str(l+1)] = np.zeros((parameters["W" + str(l+1)]).shape)
        s["db" + str(l+1)] = np.zeros((parameters["b" + str(l+1)]).shape)
    
    return v, s

In [12]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    
    # In the adam Paper by default beta1 is taken as 0.9 and beta2 as 0.999 and the epsilon as 10^(-8)
    
    """
    Update parameters using Adam
    
    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    learning_rate -- the learning rate, scalar.
    beta1 -- Exponential decay hyperparameter for the first moment estimates 
    beta2 -- Exponential decay hyperparameter for the second moment estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates

    Returns:
    parameters -- python dictionary containing your updated parameters 
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    """
    
    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    # Perform Adam update on all parameters
    for l in range(L):
        # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".
        v["dW" + str(l+1)] = beta1 * v["dW" + str(l+1)] + (1 - beta1) * grads["dW" + str(l+1)]
        v["db" + str(l+1)] = beta1 * v["db" + str(l+1)] + (1 - beta1) * grads["db" + str(l+1)]

        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".
        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)] /(1 - beta1 ** t)
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)] /(1 - beta1 ** t)

        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".
        s["dW" + str(l+1)] = beta2 * s["dW" + str(l+1)] + (1 - beta2) * (np.square(grads["dW" + str(l+1)]) )
        s["db" + str(l+1)] = beta2 * s["db" + str(l+1)] + (1 - beta2) * (np.square(grads["db" + str(l+1)]) )

        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".
        s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)] /(1 - beta2 ** t)
        s_corrected["db" + str(l+1)] = s["db" + str(l+1)] /(1 - beta2 ** t)

        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate *  v_corrected["dW" + str(l+1)] /(np.sqrt(s_corrected["dW" + str(l+1)]) + epsilon)
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * v_corrected["db" + str(l+1)] / (np.sqrt(s_corrected["db" + str(l+1)]) + epsilon)

    return parameters, v, s

In [17]:
# currently do not use loss at all, so something is probably very wrong

def train(model, optimiser):
    batch_size=32
    for steps in range(2): # TODO: please increase
        # sample batch of data
        xb, yb = get_batch('train', batch_size) # TODO: pls adapt to above fct

        # get logits and loss
        logits, loss = model.forward(xb, yb)
        print(f"loss: {loss}")

        gradient = model.backward(yb, logits)
        before_adam = model.token_embedding_table.copy()
        #print(f"before Adam: {model.token_embedding_table}")
        
        param_dict = {"weight": model.token_embedding_table}
        param_dict, v, s = update_parameters_with_adam(parameters=param_dict, grads=gradient, v=optimiser[0], s=optimiser[1], t=5)
        #optimiser.update(param=model.token_embedding_table, param_grad=gradient, param_name="gradient", cur_loss = loss)
        #optimiser.step()
        #print(f"after Adam: {model.token_embedding_table}")
        model.token_embedding_table = param_dict['weight']
        diff = before_adam - model.token_embedding_table.copy()
        print(f"Difference in weights before - after Adam: {diff}")
        

In [18]:
vocab_size = 80

my_neural_embedding = neural_embedding(vocab_size)
param_dict = {"weight": my_neural_embedding.token_embedding_table} # one entry, key is weight and value is my_neural
#optimiser = Adam(params = my_neural_embedding.token_embedding_table, lr=0.3)
optimiser_v, optimiser_s = initialize_adam(param_dict)
print(f"Optimiser v: {optimiser_v}, optimiser s: {optimiser_s}")
optimiser = [optimiser_v, optimiser_s]
train(my_neural_embedding, optimiser)
#uglies_array_youve_ever_seeeeeen = np.array([[1,2,3], [3,4,7]])
#my_input_logits, my_loss = my_neural_embedding.forward(uglies_array_youve_ever_seeeeeen, uglies_array_youve_ever_seeeeeen)

Optimiser v: {}, optimiser s: {}
loss: 37.439745740279065
Difference in weights before - after Adam: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
loss: 37.439745740279065
Difference in weights before - after Adam: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
